In [10]:
# Import necessary libraries
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import datetime
# Ignore warnings
import warnings
import talib
warnings.filterwarnings("ignore")

# read the csv file by selecting the path
path_file = "D:/Dropbox/Pastor/data/crypto-active_5min/BTC_5min.txt"
btc = pd.read_csv(path_file, header = None, sep = ",", names = ["Date", "Open", "High", "Low", "Close", "Volume"])
btc["Date"] = pd.to_datetime(btc['Date'])
btc.set_index('Date', inplace=True)


In [11]:
ohlcv_dict = {
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    }
bars1h = btc.resample('60Min').apply(ohlcv_dict).dropna()
bars1d = btc.resample('1440Min').apply(ohlcv_dict).dropna()

In [12]:
def get_last_N_timebars(bars5m, bars1h, bars1d, curr_time, lkbk):
    '''This function gets the timebars for the 5m, 1hr and 1d resolution based
    on the lookback we've specified.
    '''
   
    """Width of the 5m, 1hr, and 1d"""
    wdw5m = 9
    wdw1h = np.ceil(lkbk*15/24.)
    wdw1d = np.ceil(lkbk*15)

    """Creating the timebars based on the lookback"""
    last5m = bars5m[curr_time-timedelta(wdw5m):curr_time].iloc[-lkbk:]
    last1h = bars1h[curr_time-timedelta(wdw1h):curr_time].iloc[-lkbk:]
    last1d = bars1d[curr_time-timedelta(wdw1d):curr_time].iloc[-lkbk:]
    
    return last5m, last1h, last1d

In [13]:
# Set the current time
curr_time = datetime.datetime(2020, 5, 31, 11, 0, 0)

# Fixed the lookback period to get last time bars from the current time.
# This is also used to calculate the value of technical indicators. This is a hyperparameter that you can tweak.
lkbk = 20

# Store the last N time bars for 5mins, 1h, and 1d bars
last5m, last1h, last1d = get_last_N_timebars(
    btc, bars1h, bars1d, curr_time, lkbk)

last5m

,Open,High,Low,Close,Volume
Date,,,,,
2020-05-31 09:25:00,9575.900,9590.500,9575.700,9590.400,17.053
2020-05-31 09:30:00,9590.500,9590.500,9579.900,9584.357,59.596
2020-05-31 09:35:00,9584.395,9584.395,9577.900,9577.900,15.313
2020-05-31 09:40:00,9574.700,9577.900,9568.200,9572.500,3.444
2020-05-31 09:45:00,9569.700,9571.000,9565.415,9565.415,2.541
2020-05-31 09:50:00,9564.600,9566.200,9542.400,9542.400,85.851
2020-05-31 09:55:00,9566.100,9566.200,9558.743,9562.100,20.264
2020-05-31 10:00:00,9558.743,9558.743,9531.800,9539.900,29.563
2020-05-31 10:05:00,9539.921,9562.200,9539.921,9556.700,19.334


In [14]:
"""Adding State Variable"""
state = np.array([])
state.shape

(0,)

In [15]:
last5m_flat = last5m.iloc[-10:, :-1].values.flatten()
last5m_flat

array([9541.4  , 9542.   , 9538.   , 9539.   , 9538.087, 9538.087,
       9526.1  , 9533.132, 9533.132, 9547.574, 9527.1  , 9547.5  ,
       9547.5  , 9547.5  , 9533.3  , 9535.6  , 9535.6  , 9545.2  ,
       9535.5  , 9545.2  , 9545.2  , 9545.2  , 9535.7  , 9535.7  ,
       9535.7  , 9535.7  , 9511.6  , 9526.2  , 9526.2  , 9526.2  ,
       9520.3  , 9525.206, 9525.2  , 9525.206, 9460.592, 9469.9  ,
       9468.7  , 9494.3  , 9455.6  , 9484.9  ])

In [16]:
print('Shape of the array:', last5m_flat.shape)

Shape of the array: (40,)


# Create stationary candlestick bars
This is done by calculating Z-scores

In [17]:
"""Normalizing candlesticks for 5 mins data"""
last5m_norm = (last5m_flat-np.mean(last5m_flat))/np.std(last5m_flat)
last5m_norm

array([ 0.64136501,  0.66594686,  0.50206787,  0.54303762,  0.50563224,
        0.50563224,  0.01452787,  0.30262714,  0.30262714,  0.89431224,
        0.05549762,  0.89128048,  0.89128048,  0.89128048,  0.30951006,
        0.40374048,  0.40374048,  0.79705006,  0.3996435 ,  0.79705006,
        0.79705006,  0.79705006,  0.40783745,  0.40783745,  0.40783745,
        0.40783745, -0.57953347,  0.01862485,  0.01862485,  0.01862485,
       -0.22309667, -0.02209908, -0.0223449 , -0.02209908, -2.66931837,
       -2.28797196, -2.33713565, -1.28831011, -2.87383935, -1.67342574])

In [18]:
last5m_norm.shape

(40,)

In [19]:
"""Adding candlesticks"""
def get_normalised_bars_array(bars):
    bars = bars.iloc[-10:, :-1].values.flatten()

    """Normalizing candlesticks"""
    bars = (bars-np.mean(bars))/np.std(bars)
    return bars


"""" Append normalised candlestick bars for all granularity"""
state = np.append(state, get_normalised_bars_array(last5m))
state = np.append(state, get_normalised_bars_array(last1h))
state = np.append(state, get_normalised_bars_array(last1d))
state

array([ 0.64136501,  0.66594686,  0.50206787,  0.54303762,  0.50563224,
        0.50563224,  0.01452787,  0.30262714,  0.30262714,  0.89431224,
        0.05549762,  0.89128048,  0.89128048,  0.89128048,  0.30951006,
        0.40374048,  0.40374048,  0.79705006,  0.3996435 ,  0.79705006,
        0.79705006,  0.79705006,  0.40783745,  0.40783745,  0.40783745,
        0.40783745, -0.57953347,  0.01862485,  0.01862485,  0.01862485,
       -0.22309667, -0.02209908, -0.0223449 , -0.02209908, -2.66931837,
       -2.28797196, -2.33713565, -1.28831011, -2.87383935, -1.67342574,
        0.37593974,  0.54310346, -0.27210589,  0.06661817,  0.03082682,
        0.66904872, -0.63849211,  0.40934959,  0.40934959,  0.53623371,
       -0.56750478, -0.3888915 , -0.50796702,  0.27014572, -1.45715922,
        0.26831379,  0.26950455,  1.13390124, -0.10883503,  1.13161132,
        1.10184244,  1.13390124,  0.54539337,  0.81622149,  0.81789312,
        2.10482473,  0.24541465,  0.57745216,  0.57745216,  0.90

In [20]:
state.shape

(120,)

# Technical Indicators
We create technical indicators to the stay array. We will be using:

    1. Relative differences between two moving averages
    2. RSI
    3. Momentum
    4. Balance of Power
    5. Aroon Oscilator
    

In [21]:
# create an array that stores the technical indicators:
tech_ind = np.array([])
# lkbk is lookback period we are using 19 SMA and 12 SMA given that our lkbk is 20

"""Relative differences between two moving averages"""
sma1 = talib.SMA(last5m['Close'], lkbk-1)[-1]
sma2 = talib.SMA(last5m['Close'], lkbk-8)[-1]
tech_ind = np.append(tech_ind, (sma1-sma2)/sma2)

"""Relative Strength Index"""
tech_ind = np.append(tech_ind, talib.RSI(
    last5m['Close'], lkbk-1)[-1])

"""Momentum"""
tech_ind = np.append(tech_ind, talib.MOM(
    last5m['Close'], lkbk-1)[-1])

"""Balance of Power"""
tech_ind = np.append(tech_ind, talib.BOP(last5m['Open'],
                                         last5m['High'],
                                         last5m['Low'],
                                         last5m['Close'])[-1])
"""Aroon Oscillator"""
tech_ind = np.append(tech_ind, talib.AROONOSC(last5m['High'],
                                              last5m['Low'],
                                              lkbk-3)[-1])
tech_ind

array([ 1.35872359e-03,  2.94295653e+01, -1.05500000e+02,  4.18604651e-01,
       -1.00000000e+02])

In [22]:
def get_technical_indicators(bars):
    # Create an array to store the value of indicators
    tech_ind = np.array([])

    """Relative difference two moving averages"""
    sma1 = talib.SMA(bars['Close'], lkbk-1)[-1]
    sma2 = talib.SMA(bars['Close'], lkbk-8)[-1]
    tech_ind = np.append(tech_ind, (sma1-sma2)/sma2)

    """Relative Strength Index"""
    tech_ind = np.append(tech_ind, talib.RSI(
        bars['Close'], lkbk-1)[-1])

    """Momentum"""
    tech_ind = np.append(tech_ind, talib.MOM(
        bars['Close'], lkbk-1)[-1])

    """Balance of Power"""
    tech_ind = np.append(tech_ind, talib.BOP(bars['Open'],
                                             bars['High'],
                                             bars['Low'],
                                             bars['Close'])[-1])

    """Aroon Oscillator"""
    tech_ind = np.append(tech_ind, talib.AROONOSC(bars['High'],
                                                  bars['Low'],
                                                  lkbk-3)[-1])
    return tech_ind

In [23]:
state = np.append(state, get_technical_indicators(last5m))
state = np.append(state, get_technical_indicators(last1h))
state = np.append(state, get_technical_indicators(last1d))
state

array([ 6.41365015e-01,  6.65946864e-01,  5.02067872e-01,  5.43037620e-01,
        5.05632240e-01,  5.05632240e-01,  1.45278726e-02,  3.02627140e-01,
        3.02627140e-01,  8.94312238e-01,  5.54976205e-02,  8.91280477e-01,
        8.91280477e-01,  8.91280477e-01,  3.09510057e-01,  4.03740477e-01,
        4.03740477e-01,  7.97050057e-01,  3.99643503e-01,  7.97050057e-01,
        7.97050057e-01,  7.97050057e-01,  4.07837452e-01,  4.07837452e-01,
        4.07837452e-01,  4.07837452e-01, -5.79533471e-01,  1.86248474e-02,
        1.86248474e-02,  1.86248474e-02, -2.23096665e-01, -2.20990820e-02,
       -2.23449005e-02, -2.20990820e-02, -2.66931837e+00, -2.28797196e+00,
       -2.33713565e+00, -1.28831011e+00, -2.87383935e+00, -1.67342574e+00,
        3.75939742e-01,  5.43103456e-01, -2.72105887e-01,  6.66181748e-02,
        3.08268207e-02,  6.69048720e-01, -6.38492109e-01,  4.09349586e-01,
        4.09349586e-01,  5.36233714e-01, -5.67504778e-01, -3.88891495e-01,
       -5.07967017e-01,  

In [24]:
state.shape

(135,)

We have 5 technical indicators, for each granularity data we get 5 values. The size of the technical indicators is 15(5*3). And the total size of the state array is 135, that is (120+15)

# Time Signature
We add time of the day, day of the week and position of the state array.

# Time of the day 
To calculate the time of the day, we use hours and minutes from the current time. We divide by 24*60 to convert the time into an hour and finally normalise it.

In [25]:
# Extract hours and minutes from current time and map that into float
tm_lst = list(map(float, str(curr_time.time()).split(':')[:2]))

# Calculate time of the day
_time_of_day = (tm_lst[0]*60 + tm_lst[1])/(24*60)
print('Time of the day:', _time_of_day)

Time of the day: 0.4583333333333333


### Day of the week

We use `weekday()` property of datetime to get the day of the week as an integer. Then we divide it by 6 to normalise it.


Syntax: 
```python
DatetimeIndex.weekday()
```
 
Returns:

Integers indicating the day of the week. The day of the week with: <br>
Monday = 0, 
Tuesday = 1,
Wednesday = 2,
Thursday = 3,
Friday = 4,
Saturday = 5,
Sunday = 6

In [26]:
_day_of_week = curr_time.weekday()/6
print('Day of the Week:', _day_of_week)

Day of the Week: 1.0


In [27]:
# The position is updated using update_postion() function of Game class. 
# For the illustration purpose, we have manually set to 1.
position = 1
state = np.append(state, _time_of_day)
state = np.append(state, _day_of_week)
state = np.append(state, position)
state
#state.shape

array([ 6.41365015e-01,  6.65946864e-01,  5.02067872e-01,  5.43037620e-01,
        5.05632240e-01,  5.05632240e-01,  1.45278726e-02,  3.02627140e-01,
        3.02627140e-01,  8.94312238e-01,  5.54976205e-02,  8.91280477e-01,
        8.91280477e-01,  8.91280477e-01,  3.09510057e-01,  4.03740477e-01,
        4.03740477e-01,  7.97050057e-01,  3.99643503e-01,  7.97050057e-01,
        7.97050057e-01,  7.97050057e-01,  4.07837452e-01,  4.07837452e-01,
        4.07837452e-01,  4.07837452e-01, -5.79533471e-01,  1.86248474e-02,
        1.86248474e-02,  1.86248474e-02, -2.23096665e-01, -2.20990820e-02,
       -2.23449005e-02, -2.20990820e-02, -2.66931837e+00, -2.28797196e+00,
       -2.33713565e+00, -1.28831011e+00, -2.87383935e+00, -1.67342574e+00,
        3.75939742e-01,  5.43103456e-01, -2.72105887e-01,  6.66181748e-02,
        3.08268207e-02,  6.69048720e-01, -6.38492109e-01,  4.09349586e-01,
        4.09349586e-01,  5.36233714e-01, -5.67504778e-01, -3.88891495e-01,
       -5.07967017e-01,  